In [4]:
import requests
from bs4 import BeautifulSoup
import functools

Grabbing Webpage

In [5]:
url = "https://masteringruneterra.com/mu-table/"
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

Doing finding and stuff

In [6]:
headers = soup.select("div.grid-item:not(.versus-cell)")
headers = map(lambda x: x.text.strip(), headers)
decks = list(headers)[1:16]
print(len(decks))
print(decks)

15
['Gwen Zed (IO/SI)', 'Annie Jhin (NX/RU)', 'Elder Dragon Mordekaiser Viego (RU/SI)', "Pyke Rek'Sai (BW/SH)", 'Elder Dragon Galio Morgana (DE/RU)', 'Master Yi Nami (BW/IO)', 'Ashe LeBlanc (FR/NX)', 'Caitlyn Teemo (FR/PZ)', 'Elder Dragon Volibear (FR/RU)', 'Aurelion Sol Elder Dragon (MT/RU)', 'Karma Sett (IO/PZ)', 'Elder Dragon Morgana Shyvana (DE/RU)', 'Karma Sett (FR/IO)', 'Janna Nilah (BW/PZ)', 'Maokai Nautilus (BW/SI)']


In [7]:
def handleWinrate(winRate: str) -> str:
    winRate = winRate[:-1]  #dropping %
    winRate = float(winRate)
    winRate = winRate / 100
    winRate = round(winRate, 4)
    return winRate


In [8]:

cells = soup.select(".versus-cell > div")
rawText = map(lambda x: x.text.strip(), cells)


def getWinRateAndGames(text):
    winRate, games = text.split("\n");
    #handing winrate
    winRate = handleWinrate(winRate)

    #handling games
    games = games.split(" ")[1]
    games = int(games)
    return {"winRate": winRate, "gamesPlayed": games}


matchUps = list(map(getWinRateAndGames, rawText))
matchUps[0:10]



[{'winRate': 0.3862, 'gamesPlayed': 593},
 {'winRate': 0.6562, 'gamesPlayed': 477},
 {'winRate': 0.6667, 'gamesPlayed': 480},
 {'winRate': 0.5623, 'gamesPlayed': 377},
 {'winRate': 0.5831, 'gamesPlayed': 307},
 {'winRate': 0.5273, 'gamesPlayed': 330},
 {'winRate': 0.6317, 'gamesPlayed': 315},
 {'winRate': 0.805, 'gamesPlayed': 241},
 {'winRate': 0.6459, 'gamesPlayed': 257},
 {'winRate': 0.4938, 'gamesPlayed': 243}]

Converting to a table

In [9]:
table = {key: {} for key in decks}
muIndex = 0
for rawIndex in range(len(decks) * len(decks)):
    i = rawIndex // 15
    j = rawIndex % 15
    deck1 = decks[i]
    deck2 = decks[j]
    if i == j:  #diagonal on the match up, the deck is playing itself
        table[deck1][deck2] = {"win_rate": 0.5, "games_played": 10000}
        continue
    mu = matchUps[muIndex]
    muIndex += 1
    table[deck1][deck2] = {"win_rate": mu["winRate"], "games_played": mu["gamesPlayed"]}
print(table)

{'Gwen Zed (IO/SI)': {'Gwen Zed (IO/SI)': {'win_rate': 0.5, 'games_played': 10000}, 'Annie Jhin (NX/RU)': {'win_rate': 0.3862, 'games_played': 593}, 'Elder Dragon Mordekaiser Viego (RU/SI)': {'win_rate': 0.6562, 'games_played': 477}, "Pyke Rek'Sai (BW/SH)": {'win_rate': 0.6667, 'games_played': 480}, 'Elder Dragon Galio Morgana (DE/RU)': {'win_rate': 0.5623, 'games_played': 377}, 'Master Yi Nami (BW/IO)': {'win_rate': 0.5831, 'games_played': 307}, 'Ashe LeBlanc (FR/NX)': {'win_rate': 0.5273, 'games_played': 330}, 'Caitlyn Teemo (FR/PZ)': {'win_rate': 0.6317, 'games_played': 315}, 'Elder Dragon Volibear (FR/RU)': {'win_rate': 0.805, 'games_played': 241}, 'Aurelion Sol Elder Dragon (MT/RU)': {'win_rate': 0.6459, 'games_played': 257}, 'Karma Sett (IO/PZ)': {'win_rate': 0.4938, 'games_played': 243}, 'Elder Dragon Morgana Shyvana (DE/RU)': {'win_rate': 0.4121, 'games_played': 199}, 'Karma Sett (FR/IO)': {'win_rate': 0.7354, 'games_played': 223}, 'Janna Nilah (BW/PZ)': {'win_rate': 0.4429, 'g

Scraping overall play and winrates

In [10]:
deckCells = soup.select(".grid-container:not(:first-child):not(:last-child) > .grid-item.side-header:first-child")
len(deckCells)

15

In [11]:
from dataclasses import dataclass, asdict


@dataclass
class Archetype:
    name: str
    games_played: int
    win_rate: float
    play_rate: float


def parseDeckCell(deckCell):
    children = list(deckCell.childGenerator())
    children = [child.text for child in children]
    name = children[0]

    games_played = children[1].split(" ")[1]
    games_played = int(games_played)

    win_rate = children[2].split(" ")[1]
    win_rate = handleWinrate(win_rate)

    play_rate = children[4].split(" ")[1]
    play_rate = handleWinrate(play_rate)

    ans = Archetype(name, games_played, win_rate, play_rate)
    return ans


archeTypes = [parseDeckCell(cell) for cell in deckCells]

# Sending to DB

In [13]:
# from sqlalchemy import create_engine, text
# from dotenv import load_dotenv
# import os
# 
# load_dotenv()
# 
# db_url = os.environ.get("DB_URI")
# engine = create_engine(db_url, pool_pre_ping=True);

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres

Inserting archetypes

In [14]:
def insert_decks_and_matchups(archeTypes, table):
    with engine.begin() as connection:
        #clearing table first
        cmd = "TRUNCATE archetypes, matchups"
        connection.execute(text(cmd))
        
        #inserting decks
        for archeType in archeTypes:
            cmd = "insert into archetypes (NAME, games_played, win_rate, play_rate) VALUES (:name, :games_played, :win_rate, :play_rate)"
            connection.execute(text(cmd), asdict(archeType))
            
        #inserting matchups
        for playing_deck, opposing_decks in table.items():
            for opposing_deck, data in opposing_decks.items():
                cmd = """INSERT INTO matchups (playing, opposing, win_rate, games_played) VALUES (
                    (SELECT id from archetypes WHERE
                        archetypes.name = :playing_name),
                    (SELECT id from archetypes WHERE
                        archetypes.name = :opposing_name),
                    :win_rate,
                    :games_played
                    )
                """
                binds = {"playing_name": playing_deck, "opposing_name": opposing_deck, "win_rate": data["win_rate"],
                         "games_played": data["games_played"]}
                connection.execute(text(cmd), binds)


In [15]:
# insert_decks_and_matchups(archeTypes, table)

Writing to local file

In [16]:
table

{'Gwen Zed (IO/SI)': {'Gwen Zed (IO/SI)': {'win_rate': 0.5,
   'games_played': 10000},
  'Annie Jhin (NX/RU)': {'win_rate': 0.3862, 'games_played': 593},
  'Elder Dragon Mordekaiser Viego (RU/SI)': {'win_rate': 0.6562,
   'games_played': 477},
  "Pyke Rek'Sai (BW/SH)": {'win_rate': 0.6667, 'games_played': 480},
  'Elder Dragon Galio Morgana (DE/RU)': {'win_rate': 0.5623,
   'games_played': 377},
  'Master Yi Nami (BW/IO)': {'win_rate': 0.5831, 'games_played': 307},
  'Ashe LeBlanc (FR/NX)': {'win_rate': 0.5273, 'games_played': 330},
  'Caitlyn Teemo (FR/PZ)': {'win_rate': 0.6317, 'games_played': 315},
  'Elder Dragon Volibear (FR/RU)': {'win_rate': 0.805, 'games_played': 241},
  'Aurelion Sol Elder Dragon (MT/RU)': {'win_rate': 0.6459,
   'games_played': 257},
  'Karma Sett (IO/PZ)': {'win_rate': 0.4938, 'games_played': 243},
  'Elder Dragon Morgana Shyvana (DE/RU)': {'win_rate': 0.4121,
   'games_played': 199},
  'Karma Sett (FR/IO)': {'win_rate': 0.7354, 'games_played': 223},
  'Jann

# Writing Nodes

In [35]:
import json
output = [{"id": f"n{i}", "label": deck.name} for i, deck in enumerate(archeTypes)]
with open("data/nodes.json", "w") as f:
    json.dump(output, f, indent=4)

# Writing Edges

In [33]:
nameToId = {deck.name : f"n{i}" for i, deck in enumerate(archeTypes)}
i = 0
edgeList = []
for playingDeck, opposingDecks in table.items():
    for opposingDeck, data in opposingDecks.items():
        thisEdge = {"id": f"e{i}", "source": nameToId[playingDeck], "target": nameToId[opposingDeck], "win_rate": data["win_rate"], "games_played": data["games_played"]}
        edgeList.append(thisEdge)
        i += 1
display(edgeList[2])

{'id': 'e2',
 'source': 'n0',
 'target': 'n2',
 'win_rate': 0.6562,
 'games_played': 477}

In [34]:
with open("data/edges.json", "w") as f:
    json.dump(edgeList, f)